In [ ]:
def get_properties(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}
    
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]

    parameters = {'S_eV':{'expression':'Entropy','after':0,'field':2},
                  'S_J/molK':{'expression':'Entropy','after':1,'field':1},
                  'G_eV':{'expression':'Total free energy :','after':1,'field':4},
                  'G_kJ/molK':{'expression':'Total free energy (in kJmol-1)','after':1,'field':4},
                  'G_defect_eV':{'expression':'Final defect energy','after':0,'field':4},
                  **{f'sdlc{i}{j}':{'expression':'Static dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'hfdlc{i}{j}':{'expression':'High frequency dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'C{i}{j}':{'expression':'Elastic Constant Matrix: (Units=GPa)','after':4+i,'field':j} for i in (1,2,3,4,5,6) for j in (1,2,3,4,5,6)},
                  **{f'bulk_modulus_{i}':{'expression':'Mechanical properties :','after':5,'field':4+('Reuss', 'Voigt', 'Hill').index(i)} for i in ('Reuss', 'Voigt', 'Hill')},
                  **{f'lattice_parameter_{i}':{'expression':'Comparison of initial and final structures :','after':6+('a', 'b', 'c').index(i),'field':2} for i in ('a', 'b', 'c')}}

    for element in wanted_properties:
        expression = parameters[element]['expression']
        after = parameters[element]['after']
        field = parameters[element]['field']  
        for line_number,line in enumerate(lines_bottom):
            if expression in line: 
                properties[wanted_properties[wanted_properties.index(element)]] = lines_bottom[line_number+after].split()[field]

    return(properties)

In [ ]:
import csv

caminho = '/content/drive/My Drive/Colab Notebooks/'
wanted_properties = ['G_defect_eV','lattice_parameter_a','C11','S_eV','S_J/molK']

valor_inicial = 300
valor_final = 310
passo = 10

with open('/content/drive/My Drive/Colab Notebooks/Saida.csv','w') as arquivo:

    writer = csv.writer(arquivo)
    writer.writerow(wanted_properties)
    
    for i in range(valor_inicial,valor_final + passo,passo):
        j = float(i)
        filename = f'Li2S_v_Li_New({j}).out' #f'Li2S_v_Li_New({j}).out' f'Li2S_New({j}).out' 
        fullname = caminho + filename
        propriedades = get_properties(fullname,wanted_properties)
        writer.writerow(list(propriedades.values()))